In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import random
import copy
import time
import plotly.express as px
import matplotlib.pyplot as plt
from pyomo.environ import *
random.seed(0)
np.random.seed(0)

In [ ]:
df = pd.read_excel("Data/Indigo_Fleet.xlsx",index_col=False)

In [ ]:
class airport:
    
    def __init__(self, aport, aircraftCount_dict):
        self.aport = aport
        self.aircraftCount_dict = aircraftCount_dict

In [ ]:
class originAirport:
    
    def __init__(self, airport, deptime, depday):
        self.airport = airport
        self.deptime = deptime
        self.depday = depday

In [ ]:
class destinationAirport:
    
    def __init__(self, airport, arrtime, arrday, elapsed_time, total_prop_delay, flight_time):
        self.airport = airport
        self.arrtime = arrtime
        self.arrday = arrday
        self.elapsed_time = elapsed_time
        self.total_prop_delay = total_prop_delay
        self.flight_time = flight_time

In [ ]:
class FlightLeg:

    def __init__(self,origin,dest,flights,day,arr_delay,dep_datetime,arr_datetime,flighttime,
                 total_propagated_delay,pointer,elapsed_time, processed, firstnodeorigin, averageratings
                 , priorityIndex, priorityAssigned, cluster_id
                 , expected_demand , sum_prev_priority, per_seat
                 , best_combination_value, normalized_per_seat, fleet
                 , serviceratings = [], foodratings = [], entertainmentratings = [], comfortratings = []):
        self.origin = origin
        self.dest = dest
        self.flights = flights
        self.day = day
        self.arr_delay = arr_delay
        self.dep_datetime = self.dateTimeObject(str(dep_datetime))
        self.arr_datetime = self.dateTimeObject(str(arr_datetime))
        self.flighttime = flighttime
        self.total_propagated_delay = total_propagated_delay
        self.pointer = pointer
        self.elapsed_time = elapsed_time
        self.processed = processed
        self.firstnodeorigin = firstnodeorigin
        self.priorityIndex = priorityIndex
        self.priorityAssigned = priorityAssigned
        self.serviceratings = serviceratings
        self.foodratings = foodratings
        self.entertainmentratings = entertainmentratings
        self.comfortratings = comfortratings
        self.averageratings = averageratings
        self.cluster_id = cluster_id
        self.sum_prev_priority = sum_prev_priority
        self.best_combination_value = best_combination_value
        self.expected_demand = expected_demand
        self.per_seat = per_seat
        self.normalized_per_seat = normalized_per_seat
        self.fleet = fleet
        
    def dateTimeObject(self,String):
        return datetime.datetime.strptime(String,"%Y-%m-%d %H:%M:%S")

In [ ]:
def calculate_slack_time_seconds(Dept,Arr, day1, day2, mean_turnaround_time):
    if day1 >= day2:
        return ((Dept-Arr).total_seconds() - mean_turnaround_time)
    else:
        return -99

def check_slack_time_seconds_validity(slack_time):
    return (24.*3600)>=slack_time>=0 #Check for flights within stipulated time for connection not beyond that

def check_time_elapsed_validity_seconds(time_elapsed):
    return (0<=time_elapsed<=36*60*60)

def calculate_propagated_delay_seconds(prev_prop_delay,arrival_delay,slack_time):
    prop_delay = prev_prop_delay+arrival_delay-slack_time # when arrival delay is in seconds
#     if prop_delay>0:
#         return prop_delay
    return max(0, prop_delay)

def calculate_time_elapsed(prev_elapsed_time,Arr,Dept):
    return prev_elapsed_time+(Dept-Arr).total_seconds()

def set_priority(node_dict, count_dict):
    lbda = 0.7
    for n in range(len(node_dict)):
        node_dict[n].priorityIndex = node_dict[n].cluster_id * lbda * count_dict[node_dict[n].flights] + (1 - lbda) * (1/node_dict[n].averageratings)

In [ ]:
#Optimization master problem
def route_optimizer(stagewise_routes, prop_delay, route_strings
                    , priority_list, priority_number, sflights, assignedFlights, itr, penaltyCost
                    , priorityWeight, delayWeight, uniquePriorities):
    
    model = ConcreteModel()
    n = len(stagewise_routes)
    penaltyCost = 10000

    
    uniqueFlights = []
    uniquePriority = []
    fullPriority = []
    flightPriority = []
    multipleFlight = []
    fp_dict = {}

    for sr in stagewise_routes:
        for s in sr:
            uniqueFlights.append(s)
            
    for pl in priority_list:
        for p in pl:
            fullPriority.append(p)
            
    for f in uniqueFlights:
        if f not in fp_dict:
            fp_dict[f] = fullPriority[uniqueFlights.index(f)]
    
    uniqueFlights = set(uniqueFlights)
    
    for key, value in fp_dict.items():
        uniquePriority.append(value)
        
    m = len(sflights)
    nparray = []
    flightString = {}
    ff = 0
    for i in sflights:
        if ff not in flightString:
            flightString[ff] = []
        
        npList=[]
        cc = 0
        for j in stagewise_routes:
            if i in j:
                npList.append(1)
                flightString[ff].append(cc)
            else:
                npList.append(0)
            cc+=1
        nparray.append(npList)
        ff+=1
    varList = []
    fvarList = []
    stringFlight = {}
    flightString = {}
    
    for j in range(len(sflights)):
        fvarList.append(j)
        if j not in flightString:
            flightString[j] = []

    for i in range(n):
        if i not in stringFlight:
            stringFlight[i] = []
            
            for s in sflights:
                if s in stagewise_routes[i]:
                    stringFlight[i].append(sflights.index(s))
                    flightString[sflights.index(s)].append(i)
                    
    for i in range(n):
        varList.append(i)

    for s in sflights:
        mf = 0
        for key, value in stringFlight.items():
            if sflights.index(s) in value:
                mf+=1
        if mf > 1:
            multipleFlight.append(sflights.index(s))
        
    model.xVar = Var(varList, within = Integers, bounds = (0,1), initialize = 0)
    model.fVar = Var(fvarList, bounds = (0,1), initialize = 0)
    
    ft = 0
    if len(assignedFlights) > 0:
        model.assignedFlights_constraint = ConstraintList()
        for f in sflights:
            if f in assignedFlights:
                model.assignedFlights_constraint.add(model.fVar[ft] == 1)
            ft+=1
            
    model.numberFlights_constraint = Constraint(expr = sum(model.fVar[sflights.index(f)] for f in sflights
                                                           if f not in assignedFlights) <= priority_number)
    
    model.cover_constraint = ConstraintList()
    for i in range(m):
        a=nparray[i]
        add=0
        
        for j in range(n):
            k=a[j]*model.xVar[j]
            add=add+k
        
        model.cover_constraint.add(add <=1)

    model.airport_capacity_constraint = ConstraintList()
    model.airport_capacity_constraint.add(sum(model.xVar[i] for i in range(len(model.xVar))) <= len(model.xVar))
        
    model.route_flight_constraint = ConstraintList()
    for i in range(n):
        model.route_flight_constraint.add(model.xVar[i] >= sum(model.fVar[j] for j in stringFlight[i]
                                                               if j not in multipleFlight)/len(stringFlight[i]))
        
    model.multiple_flight_constraint = ConstraintList()
    for key, value in flightString.items():
        if key in multipleFlight:
            model.multiple_flight_constraint.add(model.fVar[key] <= sum(model.xVar[v] for v in value))

    model.flight_string_constraint = ConstraintList()
    for item, value in flightString.items():
        model.flight_string_constraint.add(sum(model.xVar[v] for v in value) >= model.fVar[item])
    
    aircraftRouting = []
    priority = []
    penaltyProd = []
    reward = []
    

    
    priority.append(sum(model.fVar[j] * uniquePriorities[j] for j in range(len(sflights))))
    for i in range(n):
        penaltyProd.append(penaltyCost * len(stagewise_routes[i]) * model.xVar[i])
        aircraftRouting.append(prop_delay[i] * model.xVar[i])
        
    model.value = Objective(expr = sum(aircraftRouting) * delayWeight - sum(priority) * priorityWeight
                            - sum(penaltyProd))
        
        
    result_obj = SolverFactory('mindtpy').solve(model, mip_solver='glpk', nlp_solver = 'ipopt', tee = True)

    sol_routes = []
    sol_delay = []
    index = []
    flt = []
    stringIndex = []
    c = 0
    for v in range(n):
        if model.xVar[v].value != 0:
            sol_routes.append(model.xVar[v].value)
            sol_delay.append(prop_delay[v])
            index.append(c)
            stringIndex.append(v)
        c+=1
        
    for f in range(len(sflights)):
        if model.fVar[f].value > 0.9:
            flt.append(sflights[f])
            
                
    return sol_routes, index, flt, sol_delay

In [ ]:
origin_dict_og = {}
destination_dict_og = {}
object_dict_og = {}
object_dict_copy = {}
object_dict_dep_sort = {}
fleet_cap_dict = {}
airport_aircraft_dict = {}
airport_cluster_dict = {}
cluster_aircraft_dict = {}
aircraftStringCount = {}
operatingCost_dict = {}
equipment = []
allAirports = []
allAircrafts = []
ogRevenue = []
flightCount = {}

#Sort the flight leg nodes as per the time.

for i,r in df.iterrows():
    ogRevenue.append(r["Per_seat"])
    allAirports.append(r["ORIGIN"])
    allAirports.append(r["DEST"])
#     allAircrafts.append(r["TAIL_NUM"][-2:])
    if r["FLIGHTS"] not in flightCount:
        flightCount[r["FLIGHTS"]] = 1
    elif r["FLIGHTS"] in flightCount:
        flightCount[r["FLIGHTS"]]+=1
    if r["ORIGIN"] not in aircraftStringCount:
        aircraftStringCount[r["ORIGIN"]] = []
    origin_dict_og.update({i : originAirport(r["ORIGIN"], r["CRS_DEP_DATETIME"], r["DAY_OF_MONTH"])})
    destination_dict_og.update({i : destinationAirport(r["DEST"], r["CRS_ARR_DATETIME"], r["DAY_OF_MONTH"],
                                                       timedelta(days=0,hours=0,minutes=0,seconds=0), 0.0, 0)})

    if r["CRS_DEP_DATETIME"] not in object_dict_dep_sort:
        object_dict_dep_sort[r["CRS_DEP_DATETIME"]] = []
        object_dict_dep_sort[r["CRS_DEP_DATETIME"]].append(FlightLeg(r["ORIGIN"],r["DEST"],r["FLIGHTS"],r["DAY_OF_MONTH"],
                                                                     r["ARR_DELAY_NEW"], r["CRS_DEP_DATETIME"],r["CRS_ARR_DATETIME"]
                                                                     ,0,0.0,None,0.0, 0, None, 0, 0, 0, r["CLUSTER_ID"]
                                                                     ,r["EXPECTED_DEMAND"], 0, r["Per_seat"], np.inf, r["Per_seat"]
                                                                     ,r["Fleet"], [], [], [], []))    
    else:
        object_dict_dep_sort[r["CRS_DEP_DATETIME"]].append(FlightLeg(r["ORIGIN"],r["DEST"],r["FLIGHTS"],r["DAY_OF_MONTH"],
                                                                     r["ARR_DELAY_NEW"], r["CRS_DEP_DATETIME"],r["CRS_ARR_DATETIME"]
                                                                     ,0,0.0,None,0.0, 0, None, 0, 0, 0, r["CLUSTER_ID"]
                                                                     ,r["EXPECTED_DEMAND"], 0, r["Per_seat"], np.inf, r["Per_seat"]
                                                                     ,r["Fleet"], [], [], [], []))
            

sorted_timestamp = sorted(object_dict_dep_sort)
allAirports = list(set(allAirports))
allAircrafts = list(set(allAircrafts))

In [ ]:
# Sorting the object dictionary as per the time stamps
c = 0

for s in sorted_timestamp:
    for v in object_dict_dep_sort[s]:
        object_dict_og.update({c : v})
        object_dict_copy.update({c : v})
        c+=1

In [ ]:
def give_priority(node_dict):
    maxRating = 0.0
    for n in range(len(node_dict)):
        
        if node_dict[n].cluster_id == 1:
            for rr in range(random.randint(40,100)):
                node_dict[n].serviceratings.append(random.randint(0,5))

            for rr in range(random.randint(40,100)):
                node_dict[n].foodratings.append(random.randint(0,5))


            for rr in range(random.randint(40,100)):
                node_dict[n].entertainmentratings.append(random.randint(0,5))


            for rr in range(random.randint(40,100)):
                node_dict[n].comfortratings.append(random.randint(0,5))

            lnc = len(node_dict[n].comfortratings)
            lnf = len(node_dict[n].foodratings)
            lns = len(node_dict[n].serviceratings)
            lne = len(node_dict[n].entertainmentratings)

            node_dict[n].averageratings = (sum(node_dict[n].comfortratings)/lnc + sum(node_dict[n].foodratings)/lnf
                                           + sum(node_dict[n].serviceratings)/lns
                                           + sum(node_dict[n].entertainmentratings)/lne)/4
            node_dict[n].pre_averageratings = node_dict[n].averageratings
            if maxRating < node_dict[n].averageratings:
                maxRating = node_dict[n].averageratings
        
        if node_dict[n].cluster_id == 2:
            for rr in range(random.randint(40,100)):
                node_dict[n].serviceratings.append(random.randint(0,5))


            for rr in range(random.randint(40,100)):
                node_dict[n].foodratings.append(random.randint(0,5))


            for rr in range(random.randint(40,100)):
                node_dict[n].entertainmentratings.append(random.randint(0,5))

            for rr in range(random.randint(40,100)):
                node_dict[n].comfortratings.append(random.randint(0,5))

            lnc = len(node_dict[n].comfortratings)
            lnf = len(node_dict[n].foodratings)
            lns = len(node_dict[n].serviceratings)
            lne = len(node_dict[n].entertainmentratings)

            node_dict[n].averageratings = (sum(node_dict[n].comfortratings)/lnc + sum(node_dict[n].foodratings)/lnf
                                           + sum(node_dict[n].serviceratings)/lns
                                           + sum(node_dict[n].entertainmentratings)/lne)/4
            node_dict[n].pre_averageratings = node_dict[n].averageratings
            if maxRating < node_dict[n].averageratings:
                maxRating = node_dict[n].averageratings
        
        if node_dict[n].cluster_id == 3:
            for rr in range(random.randint(40,100)):
                node_dict[n].serviceratings.append(random.randint(0,5))

            for rr in range(random.randint(40,100)):
                node_dict[n].foodratings.append(random.randint(0,5))

            for rr in range(random.randint(40,100)):
                node_dict[n].entertainmentratings.append(random.randint(0,5))

            for rr in range(random.randint(40,100)):
                node_dict[n].comfortratings.append(random.randint(0,5))


            lnc = len(node_dict[n].comfortratings)
            lnf = len(node_dict[n].foodratings)
            lns = len(node_dict[n].serviceratings)
            lne = len(node_dict[n].entertainmentratings)

            node_dict[n].averageratings = (sum(node_dict[n].comfortratings)/lnc + sum(node_dict[n].foodratings)/lnf
                                           + sum(node_dict[n].serviceratings)/lns
                                           + sum(node_dict[n].entertainmentratings)/lne)/4
            node_dict[n].pre_averageratings = node_dict[n].averageratings
            if maxRating < node_dict[n].averageratings:
                maxRating = node_dict[n].averageratings
        
        if node_dict[n].cluster_id == 4:
            for rr in range(random.randint(40,100)):
                node_dict[n].serviceratings.append(random.randint(0,5))

            for rr in range(random.randint(40,100)):
                node_dict[n].foodratings.append(random.randint(0,5))

            for rr in range(random.randint(40,100)):
                node_dict[n].entertainmentratings.append(random.randint(0,5))

            for rr in range(random.randint(40,100)):
                node_dict[n].comfortratings.append(random.randint(0,5))

            lnc = len(node_dict[n].comfortratings)
            lnf = len(node_dict[n].foodratings)
            lns = len(node_dict[n].serviceratings)
            lne = len(node_dict[n].entertainmentratings)

            node_dict[n].averageratings = (sum(node_dict[n].comfortratings)/lnc + sum(node_dict[n].foodratings)/lnf
                                           + sum(node_dict[n].serviceratings)/lns
                                           + sum(node_dict[n].entertainmentratings)/lne)/4
            node_dict[n].pre_averageratings = node_dict[n].averageratings
            if maxRating < node_dict[n].averageratings:
                maxRating = node_dict[n].averageratings
        
        if node_dict[n].cluster_id == 5:
            for rr in range(random.randint(40,100)):
                node_dict[n].serviceratings.append(random.randint(0,5))

            for rr in range(random.randint(40,100)):
                node_dict[n].foodratings.append(random.randint(0,5))

            for rr in range(random.randint(40,100)):
                node_dict[n].entertainmentratings.append(random.randint(0,5))

            for rr in range(random.randint(40,100)):
                node_dict[n].comfortratings.append(random.randint(0,5))

            lnc = len(node_dict[n].comfortratings)
            lnf = len(node_dict[n].foodratings)
            lns = len(node_dict[n].serviceratings)
            lne = len(node_dict[n].entertainmentratings)

            node_dict[n].averageratings = (sum(node_dict[n].comfortratings)/lnc + sum(node_dict[n].foodratings)/lnf
                                           + sum(node_dict[n].serviceratings)/lns
                                           + sum(node_dict[n].entertainmentratings)/lne)/4
            node_dict[n].pre_averageratings = node_dict[n].averageratings
            if maxRating < node_dict[n].averageratings:
                maxRating = node_dict[n].averageratings
    
    for n in range(len(node_dict)):
        node_dict[n].averageratings = node_dict[n].averageratings/5

    for n in range(len(node_dict)):
        if node_dict[n].cluster_id == 1:
            number = random.uniform(0.1, 1)
            node_dict[n].averageratings = round(number, 3)
            
        if node_dict[n].cluster_id == 2:
            number = random.uniform(0.1, 1)
            node_dict[n].averageratings = round(number, 3)
            
        if node_dict[n].cluster_id == 3:
            number = random.uniform(0.1, 1)
            node_dict[n].averageratings = round(number, 3)
            
        if node_dict[n].cluster_id == 4:
            number = random.uniform(0.001, 1)
            node_dict[n].averageratings = round(number, 3)
            
        if node_dict[n].cluster_id == 5:
            number = random.uniform(0.001, 1)
            node_dict[n].averageratings = round(number, 3)
        

def set_priority(node_dict, count_dict, maxAverageRating):
    lbda = 0.7
    for n in range(len(node_dict)):
        nodeAvg = (1/node_dict[n].averageratings)
        node_dict[n].priorityIndex = lbda * (node_dict[n].cluster_id / 5)  + (1 - lbda) * nodeAvg/maxAverageRating
        
def normalize_priority(node_dict):
    maxPriority = 0.0
    for n in range(len(node_dict)):
        if maxPriority < node_dict[n].priorityIndex:
            maxPriority = node_dict[n].priorityIndex
    for n in range(len(node_dict)):
        node_dict[n].priorityIndex = node_dict[n].priorityIndex/maxPriority
        
def normalize_average_ratings(node_dict):
    maxAverageRating = 0.0
    for n in range(len(node_dict)):
        if maxAverageRating < (1/node_dict[n].averageratings):
            maxAverageRating = (1/node_dict[n].averageratings)
    return (maxAverageRating)


give_priority(object_dict_copy)
maxAverageRating = normalize_average_ratings(object_dict_copy)
set_priority(object_dict_copy, flightCount, maxAverageRating)

pi_list = []
for o in range(len(object_dict_copy)):
    pi_list.append(object_dict_copy[o].priorityIndex)

max_pi = max(pi_list)
for o in range(len(object_dict_copy)):
    object_dict_copy[o].priorityIndex = object_dict_copy[o].priorityIndex/max_pi

In [ ]:
#To verify if the node is first node
def if_first_node(node, node_dict):
    
    res = False # Set default value of the node as not the first node
    c = 0
    
    for i in range(len(object_dict_og)):
        
        if i in node_dict:
        
            if node.origin == node_dict[i].dest:

                # Check if the departure time of the node is the least among all the nodes
                if node.dep_datetime > node_dict[i].arr_datetime:
                    if node.day >= node_dict[i].day:
        #                 if node.origin == node_dict[i].dest:
                        c += 1
                        break
                
    if c == 0: # Even if 1 node found satisfying the above conditions
        
        res = True
        node.processed = 1
        node.pointer = 'dummy_start'
        node.sum_priority_idx = node.cluster_id
        
    return res

In [ ]:
def path_constructor(node_dict, dummy_list, dummy_str_list):
    
    route_list = []
    route_list_str = []
    priority_list = []
    time_list_str = []
    demand_list = []
    revenue_list = []
    normalized_revenue_list = []
    priorityDict = {}
    
    for i in node_dict:
        dummy_route = []
        dummy_route_str = []
        dummy_priority_list = []
        dummy_time_str = []
        dummy_demand_list = []
        dummy_revenue_list = []
        dummy_normalized_revenue_list = []
        
        if i in dummy_list:
            pointer = node_dict[i].pointer
            subroute_str = str(node_dict[i].origin)+'-'+str(node_dict[i].dest)
            subroute_time_str = str(node_dict[i].origin) + '-' + str(node_dict[i].dep_datetime)+'-'+str(node_dict[i].dest)  + '-' + str(node_dict[i].arr_datetime)
            dummy_route.append(i)
            dummy_route_str.append(subroute_str)
            dummy_time_str.append(subroute_time_str)
            dummy_priority_list.append(node_dict[i].priorityIndex)
            if i not in priorityDict:
                priorityDict[i] = node_dict[i].priorityIndex

            while pointer != 'dummy_start':
                if pointer != 'dummy_start':
                    if pointer not in priorityDict:
                        priorityDict[pointer] = node_dict[pointer].priorityIndex
                dummy_route.append(pointer)
                subroute_str = str(node_dict[pointer].origin)+'-'+str(node_dict[pointer].dest)
                subroute_time_str = str(node_dict[pointer].origin) + '-' + str(node_dict[pointer].dep_datetime)+'-'+str(node_dict[pointer].dest) + '-' + str(node_dict[pointer].arr_datetime)
                dummy_route_str.append(subroute_str)
                dummy_time_str.append(subroute_time_str)
                dummy_priority_list.append(node_dict[pointer].priorityIndex)
                pointer = node_dict[pointer].pointer
        
        if len(dummy_route) > 0:
            dummy_route.reverse()
            dummy_route_str.reverse()
            dummy_priority_list.reverse()
            dummy_time_str.reverse()

            route_list.append(dummy_route)
            route_list_str.append(dummy_route_str)
            priority_list.append(dummy_priority_list)
            time_list_str.append(dummy_time_str)
        
    return (route_list, route_list_str, priority_list, time_list_str, priorityDict)

In [ ]:
def get_best_feasible_link(node, node_dict, gamma2):
    
    mean_turnaround_time = 30*60
    curr_node_origin = node.origin
    best_prop_delay = np.inf
    best_prev_obj_node_idx = -99
    best_sum_priority_idx = 0
    best_comb_propdelay_priority = np.inf
    gamma1 = 1
    gamma2 = gamma2
    
    for node_idx in np.arange(len(object_dict_og)):
        if node_idx in node_dict:
            if node_dict[node_idx].processed == 1:
                dest = node_dict[node_idx].dest
                
                if node.day >= node_dict[node_idx].day:
                    if node.dep_datetime > node_dict[node_idx].arr_datetime:
                        if curr_node_origin == dest:
                            prev_elapsed_time = node_dict[node_idx].elapsed_time
                            time_elapsed = calculate_time_elapsed(prev_elapsed_time,node_dict[node_idx].arr_datetime,
                                                                  node.dep_datetime)

                            if check_time_elapsed_validity_seconds(time_elapsed):
                                slack_time = calculate_slack_time_seconds(node.dep_datetime ,node_dict[node_idx].arr_datetime
                                                                          , node.day, node_dict[node_idx].day, mean_turnaround_time)

                                if check_slack_time_seconds_validity(slack_time):
                                    node.elapsed_time = time_elapsed
                                    arrival_delay = node_dict[node_idx].arr_delay
                                    prev_prop_delay = node_dict[node_idx].total_propagated_delay
                                    prop_delay = calculate_propagated_delay_seconds(prev_prop_delay, arrival_delay, slack_time)
                                    comb_propdelay_priority = gamma1 * prop_delay - gamma2 * (node_dict[node_idx].sum_priority_idx + node.cluster_id)

                                    if comb_propdelay_priority < best_comb_propdelay_priority:
                                        best_comb_propdelay_priority = comb_propdelay_priority
                                        best_prev_obj_node_idx = node_idx
                                        best_prop_delay = prop_delay
                                        best_sum_priority_idx = node_dict[node_idx].sum_priority_idx + node.cluster_id
    
    return best_prop_delay, best_prev_obj_node_idx, best_sum_priority_idx

In [ ]:
first_node_origin = []
first_node_list = []
for i in range(len(object_dict_og)):
    if i in object_dict_copy:
        res = if_first_node(object_dict_copy[i], object_dict_copy)
        if res:
            object_dict_copy[i].elapsed_time = (object_dict_copy[i].arr_datetime - object_dict_copy[i].dep_datetime).total_seconds()
            first_node_origin.append(object_dict_copy[i].origin)
            first_node_list.append(i)

In [ ]:
final_string_list = []
final_string_str = []
final_priority_list = []
otp_dict = {}
f_assigned = []
if "5 Mins" not in otp_dict:
    otp_dict["5 Mins"] = []
if "15 Mins" not in otp_dict:
    otp_dict["15 Mins"] = []
if "30 Mins" not in otp_dict:
    otp_dict["30 Mins"] = []
if ">30 Mins" not in otp_dict:
    otp_dict[">30 Mins"] = []

In [ ]:
t0 = time.process_time()
fl_list = []
finalDelay = []
priorityWeight = 1
delayWeight = 1
priority_number_global = int(0.25 * len(object_dict_og))
priority_number = priority_number_global
penaltyCost = 1000
gamma2 = 0
sol = []
s_delay = []
flightAssigned = []
sol.append(-99)
tsearch = []
topt = []
tpath = []
tlink = []
stringLen = []
itr = 1
dispPriorityList = []
dispPathList = []
while len(sol) != 0:
    ts1 = time.process_time()
    first_node_tuple = []
    first_node_list = []
    for i in range(len(object_dict_og)):
        if i in object_dict_copy:
            res = if_first_node(object_dict_copy[i], object_dict_copy)
            if res:
                first_node_tuple.append((i, object_dict_copy[i].origin))
                first_node_list.append(i)

    temp_dummy_list = []
    dummy_terminal_node_list = []
    dummy_terminal_node_str_list = []

    tl1 = time.process_time()
    for i in range(len(object_dict_og)):
        
        if i in object_dict_copy:

            if object_dict_copy[i].processed == 0:
                best_prop_delay, best_prev_node_idx, best_sum_priority_idx = get_best_feasible_link(object_dict_copy[i], object_dict_copy, gamma2)
                if best_prev_node_idx != -99 and best_prop_delay != np.inf and best_sum_priority_idx!= 0:
                    object_dict_copy[i].pointer = best_prev_node_idx
                    object_dict_copy[i].total_propagated_delay = best_prop_delay
                    object_dict_copy[i].processed = 1
                    object_dict_copy[i].sum_priority_idx = best_sum_priority_idx

            if object_dict_copy[i].processed == 1:
                if object_dict_copy[i].pointer != 'dummy_start':
                    for f in first_node_tuple:
                        if object_dict_copy[i].pointer == f[0]:
                            object_dict_copy[i].firstnodeorigin = f[1]

                    if object_dict_copy[i].firstnodeorigin == None:
                        object_dict_copy[i].firstnodeorigin = object_dict_copy[object_dict_copy[i].pointer].firstnodeorigin


    tl2 = time.process_time()
    tlink.append(tl2 - tl1)

    all_nodes_idx = object_dict_copy.keys()
    all_pointer_idx = []
    start_pointer_idx = []

    for i in range(len(object_dict_og)):
        
        if i in object_dict_copy:
            
            if object_dict_copy[i].processed == 1:
                if object_dict_copy[i].pointer != 'dummy_start':
                    all_pointer_idx.append(object_dict_copy[i].pointer)

                if object_dict_copy[i].pointer == 'dummy_start':
                    start_pointer_idx.append(i)

    for a in all_nodes_idx:

        if a not in all_pointer_idx:
            temp_dummy_list.append(a)
    for d in temp_dummy_list:
        if d not in start_pointer_idx:
            if object_dict_copy[d].processed == 1:
                dummy_terminal_node_list.append(d)
                dummy_terminal_node_str_list.append(object_dict_copy[d].origin+'-'+object_dict_copy[d].dest)

    
    tp1 = time.process_time()
    path_list, path_list_str, priority_list, time_list, priorityDict = path_constructor(object_dict_copy
                                                                                        , dummy_terminal_node_list
                                                                                        , dummy_terminal_node_str_list)
    
    tp2 = time.process_time()
    tpath.append(tp2 - tp1)
    
    prop_delay_list = []
    sflights = []
    assignedFlights = []
    uniquePriorities = []
    
    for pl in path_list:
        for p in pl:
            if p not in sflights:
                sflights.append(p)
            if p not in assignedFlights
                if object_dict_copy[p].priorityAssigned == 1:
                    assignedFlights.append(p)
                    
    for s in sflights:
        uniquePriorities.append(priorityDict[s])

    for i in range(len(path_list)):
        prop_delay_list.append(object_dict_copy[path_list[i][len(path_list[i]) - 1]].total_propagated_delay)
    
    orgCounter = []
    if len(path_list) > 0:
        to1 = time.process_time()
        flightsFed = []
        for pl in path_list:
            for p in pl:
                if p not in flightsFed:
                    flightsFed.append(p)
        stringLen.append(len(path_list))
        sol, idx, flt, sdelay = route_optimizer(path_list, prop_delay_list, path_list_str, priority_list
                                                , priority_number, sflights, assignedFlights, itr, penaltyCost
                                                , priorityWeight, delayWeight, uniquePriorities)
        
        for i in idx:
            dispPriorityList.append(priority_list[i])
            dispPathList.append(path_list[i])
        to2 = time.process_time()
        topt.append(to2-to1)
        itr+=1
        finalDelay.append(sum(sdelay))
        if len(flt) > 0:
            for fll in flt: 
                f_assigned.append(fll)

        for s in sdelay:
            s_delay.append(s)

        sol_path = []
        flightsAfterselection = []
        for i in idx:
            sol_path.append(time_list[i])
            for pl in path_list[i]:
                if pl not in flightsAfterselection:
                    flightsAfterselection.append(pl)
        
        ff = 0
        while ff < len(flt):
            fl = flt[ff]
            if object_dict_copy[fl].priorityAssigned != 1:
                object_dict_copy[fl].priorityAssigned = 1
                ff+=1
            elif object_dict_copy[fl].priorityAssigned == 1:
                flt.remove(fl)
            fl_list.append(fl)


        priority_number -= len(flt)
        for ob in range(len(object_dict_og)):
            if ob in object_dict_copy:
                if object_dict_copy[ob].priorityAssigned == 1:
                    fla = str(object_dict_copy[ob].origin) + str("-") + str(object_dict_copy[ob].dest + str("-") + str(object_dict_copy[ob].arr_datetime) + str("-") + str(object_dict_copy[ob].dep_datetime))
                    flightAssigned.append(fla)
                    
        if len(sol) > 0:
            for i in idx:
                # print("Printing path solution: ", path_list[i])
                for j in path_list[i]:
                    if j in object_dict_copy:
                        if object_dict_copy[j].total_propagated_delay <= 5*60:
                            otp_dict["5 Mins"].append(j)
                        
                        if object_dict_copy[j].total_propagated_delay <= 15*60 and object_dict_copy[j].total_propagated_delay >= 5*60:
                            otp_dict["15 Mins"].append(j)

                        if object_dict_copy[j].total_propagated_delay <= 30*60 and object_dict_copy[j].total_propagated_delay >= 15*60:
                            otp_dict["30 Mins"].append(j)

                        if object_dict_copy[j].total_propagated_delay >= 30*60:
                            otp_dict[">30 Mins"].append(j)

                        if j in first_node_list:
                            orgCounter.append((object_dict_copy[j].origin, object_dict_copy[path_list[i][-1]].dest))
                          
                        del object_dict_copy[j]
        
    if len(path_list) == 0:
        sol = []
    
    if len(path_list) > 0:
        for i in idx:
            final_string_list.append(path_list[i])
            final_string_str.append(path_list_str[i])
            final_priority_list.append(priority_list[i])
        
    for i in range(len(object_dict_og)):
        if i in object_dict_copy:
            object_dict_copy[i].pointer = None
            object_dict_copy[i].processed = 0
            object_dict_copy[i].elapsed_time = 0.0
            object_dict_copy[i].total_propagated_delay = 0.0
        
    ts2 = time.process_time()
    tsearch.append(ts2 - ts1)

t1 = time.process_time()

print("Time elapsed: ", t1-t0)

In [ ]:
print("Printing final_string_list")
print(final_string_list)
print("Printing final_string_str")
print(final_string_str)
print("Printing s_delay")
print(s_delay)
print("Printing fl_list")
print(fl_list)
times = []
priority_list = []
for fs in final_string_list:
    temp = []
    pr_temp = []
    for f in fs:
        start_dt = object_dict_og[f].dep_datetime
        end_dt = object_dict_og[f].arr_datetime

        start_time = start_dt.strftime("%H:%M")
        end_time = end_dt.strftime("%H:%M")
        temp.append(str(start_time) + str("-") + str(end_time))
        pr_temp.append(object_dict_og[f].priorityIndex)
    times.append(temp)
    priority_list.append(pr_temp)
print(times)
tups = []
counter = 0
for fs in final_string_str:
    temp = []
    cc = 0
    for f in fs:
        temp.append((f.split('-')[0], f.split('-')[1], times[counter][cc].split("-")[0]
                     , times[counter][cc].split("-")[1]
                     , object_dict_og[final_string_list[counter][cc]].arr_delay))
        cc+=1
    counter+=1
    tups.append(temp)
print(tups)
print(priority_list)